In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.9.1
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.23.0
pandas 1.4.3
sklearn 1.1.1
tensorflow 2.9.1
keras.api._v2.keras 2.9.0


# 把python的函数变为图的目的是为了提高程序的运行速度

In [2]:
# tf.function and auto-graph.
# 自己实现一下elu激活函数,如果scale不为1，那就是selu
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    # return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))
    return scale * tf.where(is_positive, z, alpha * (tf.math.exp(z)-1))

print(scaled_elu(tf.constant(-3.)))
print('-'*50)
print(scaled_elu(tf.constant([-3., -2.5])))
print('-'*50)

# 通过tf.function可以把py的函数变为图实现的函数
scaled_elu_tf = tf.function(scaled_elu)  # 此时caled_elu_tf就是图
print(scaled_elu_tf(tf.constant(-3.)))
print('-'*50)
print(scaled_elu_tf(tf.constant([-3., -2.5])))
print('-'*50)

# 可以通过这种方式找回原来的py函数
print(scaled_elu_tf.python_function is scaled_elu)

print(scaled_elu)
print(scaled_elu_tf)  # tf的函数的执行效率比较高

tf.Tensor(-0.95021296, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
--------------------------------------------------
tf.Tensor(-0.95021296, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
--------------------------------------------------
True
<function scaled_elu at 0x0000019FD9F25CA0>


In [7]:
# 测试性能，100万个数
test = tf.random.normal((1000, 1000))

In [8]:
%timeit scaled_elu(test)

321 µs ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%timeit scaled_elu_tf(test)

361 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# 通过装饰器把函数变为图

In [17]:
# 1 + 1/2 + 1/2^2 + ... + 1/2^n
# 加了@tf.function装饰后就变为图结果，但是输入类型上不会有变化
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2)
print('-'*50)
print(converge_to_2(20))

--------------------------------------------------
tf.Tensor(1.9999981, shape=(), dtype=float32)


In [11]:
# 如何看tf的图的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [12]:
# 传普通py函数,返回的是tf图的代码
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        is_positive = ag__.converted_call(ag__.ld(tf).greater_equal, (ag__.ld(z), 0.0), None, fscope)
        try:
            do_return = True
            retval_ = ag__.ld(scale) * ag__.converted_call(ag__.ld(tf).where, (ag__.ld(is_positive), ag__.ld(z), ag__.ld(alpha) * (ag__.converted_call(ag__.ld(tf).math.exp, (ag__.ld(z),), None, fscope) - 1)), None, fscope)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

In [18]:
# to_code函数的输入是module,class, method, function, traceback, frame, or code object.不能是tf function
display_tf_code(converge_to_2.python_function)

```python
def tf__converge_to(n_iters):
    with ag__.FunctionScope('converge_to_2', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        total = ag__.converted_call(ag__.ld(tf).constant, (0.0,), None, fscope)
        increment = ag__.converted_call(ag__.ld(tf).constant, (1.0,), None, fscope)

        def get_state():
            return (total, increment)

        def set_state(vars_):
            nonlocal total, increment
            (total, increment) = vars_

        def loop_body(itr):
            nonlocal total, increment
            _ = itr
            total = ag__.ld(total)
            total += increment
            increment = ag__.ld(increment)
            increment /= 2.0
        _ = ag__.Undefined('_')
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n_iters),), None, fscope), None, loop_body, get_state, set_state, ('total', 'increment'), {'iterate_names': '_'})
        try:
            do_return = True
            retval_ = ag__.ld(total)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

In [20]:
# tf要把变量定义在函数外面，不能放里边
var = tf.Variable(0.)

@tf.function
def add_21():
    return var.assign_add(21) # += ,0+21

print(add_21())

tf.Tensor(20.0, shape=(), dtype=float32)


In [29]:
# cube计算立方，py是泛型设计，通过input_signature加类型限制可以防止调错
# @tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
@tf.function
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

print('-'*50)
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
--------------------------------------------------
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [30]:
# @tf.function py func -> tf graph
# get_concrete_function -> 给tf.function add input signature -> SavedModel

cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)
print(cube)

try:
    print(cube_func_int32(tf.constant([1, 2, 3])))
except Exception as ex:
    print(ex)

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [31]:
# 看原来函数和新生成的是否一致
# print(cube_func_int32 is cube.get_concrete_function())
print(cube.get_concrete_function(tf.TensorSpec([None], tf.int32)))
print(cube_func_int32)
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([None], tf.int32)))

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)
ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)
True


In [32]:
print(cube_func_int32)
cube_func_int32.graph

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)


In [33]:
# 图定义的一些操作
cube_func_int32.graph.get_operations()

[<tf.Operation 'z' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [34]:
pow_op = cube_func_int32.graph.get_operations()[0]
print(pow_op)

name: "z"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "z"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
    }
  }
}



In [35]:
print(list(pow_op.inputs))
print('-'*50)
print(list(pow_op.outputs))

[]
--------------------------------------------------
[<tf.Tensor 'z:0' shape=(None,) dtype=int32>]


In [36]:
cube_func_int32.graph.get_operation_by_name("z")

<tf.Operation 'z' type=Placeholder>

In [37]:
cube_func_int32.graph.get_tensor_by_name("z:0")

<tf.Tensor 'z:0' shape=(None,) dtype=int32>

In [38]:
# 图信息
cube_func_int32.graph.as_graph_def()

node {
  name: "z"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "z"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "z"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 1087
}